In [1]:
import numpy as np
import sys, os
sys.path.append('../shared/utilsGRF/')
from IPython.display import display
import writescripts 

In [2]:
edges=[(1,'a1',2),(2,'a2',3),(3,'a3',1),
       (4,'epsilon1',5),(5,'epsilon2',6),(6,'epsilon3',4),
      (1,'kb1-x',4),(4,'ku1',1),(2,'kb2-x',5),(5,'ku2',2),(3,'kb3-x',6),(6,'ku3',3)]

MTTfolder='../shared/utilsGRF' #folder with MTT.py 
basename='graph'

parlist=[x[1] for x in edges]
parlist=list(set(parlist))


list(set(parlist))

for pnum,par in enumerate(parlist):
    if '-x' in par:
        parlist[pnum]=parlist[pnum].replace('-x','')
print(parlist)

obj=writescripts.PrepareFilesNoneq(edgelist=edges,varGRF='x',concvars=['x'],parlist=parlist,MTTfolder=MTTfolder,graphbasename=basename)
obj.write_execute_parse()

obj.simpify_rhos()

['a3', 'a1', 'ku1', 'epsilon2', 'a2', 'ku2', 'epsilon3', 'kb1', 'kb2', 'epsilon1', 'ku3', 'kb3']
removing  ../shared/utilsGRF/graph-3.txt
removing  ../shared/utilsGRF/graph-5.txt
removing  ../shared/utilsGRF/graph-6.txt
removing  ../shared/utilsGRF/graph-1.txt
removing  ../shared/utilsGRF/graph-4.txt
removing  ../shared/utilsGRF/graph.txt
removing  ../shared/utilsGRF/graph-parsed.txt
removing  ../shared/utilsGRF/graph-2.txt
executing MTT
0


In [3]:
import sympy
from sympy.parsing.sympy_parser import parse_expr

In [4]:
sympy.init_printing()

In [5]:
for label in parlist:
    #print(label)
    exec("%s=sympy.symbols('%s')"%(label,label))
for var in ['x']:
    exec("%s=sympy.symbols('%s')"%(var,var))
a,alpha1,alpha2,alpha3,kb,ku=sympy.symbols('a,alpha_1,alpha_2,alpha_3,k_b,k_u')
all_rhos_sympy=[]
for rho in obj.all_rhos:
    expr=parse_expr(rho.split("=")[1])
    expr_simple=expr.subs({a1:a,a2:a,a3:a,epsilon1:alpha1*a,epsilon2:alpha2*a,epsilon3:alpha3*a,kb1:kb,kb2:kb,kb3:kb,ku1:ku,ku2:ku,ku3:ku})
    all_rhos_sympy.append(expr_simple)

In [6]:
for i in range(6):
    print("Trees for node ",i+1)
    for tree in sympy.Add.make_args(all_rhos_sympy[i]):
        print(tree)
    print()

Trees for node  1
a**2*k_u**3
a**3*alpha_1*k_u**2
a**3*alpha_2*k_u**2
a**3*alpha_3*k_u**2
a**4*alpha_1*alpha_2*k_u
a**4*alpha_1*alpha_3*k_u
a**4*alpha_2*alpha_3*k_u
a**2*alpha_2*k_b*k_u**2*x
a**2*alpha_3*k_b*k_u**2*x
a**3*alpha_1*alpha_2*k_b*k_u*x
a**2*alpha_2*alpha_3*k_b**2*k_u*x**2
2*a**3*alpha_2*alpha_3*k_b*k_u*x

Trees for node  2
a**2*k_u**3
a**3*alpha_1*k_u**2
a**3*alpha_2*k_u**2
a**3*alpha_3*k_u**2
a**4*alpha_1*alpha_2*k_u
a**4*alpha_1*alpha_3*k_u
a**4*alpha_2*alpha_3*k_u
a**2*alpha_1*k_b*k_u**2*x
a**2*alpha_3*k_b*k_u**2*x
a**2*alpha_1*alpha_3*k_b**2*k_u*x**2
a**3*alpha_2*alpha_3*k_b*k_u*x
2*a**3*alpha_1*alpha_3*k_b*k_u*x

Trees for node  3
a**2*k_u**3
a**3*alpha_1*k_u**2
a**3*alpha_2*k_u**2
a**3*alpha_3*k_u**2
a**4*alpha_1*alpha_2*k_u
a**4*alpha_1*alpha_3*k_u
a**4*alpha_2*alpha_3*k_u
a**2*alpha_1*k_b*k_u**2*x
a**2*alpha_2*k_b*k_u**2*x
a**2*alpha_1*alpha_2*k_b**2*k_u*x**2
a**3*alpha_1*alpha_3*k_b*k_u*x
2*a**3*alpha_1*alpha_2*k_b*k_u*x

Trees for node  4
a**2*k_b*k_u**2*x
a**2*al

In [7]:
phi1=all_rhos_sympy[0]
phi2=all_rhos_sympy[1]
phi3=all_rhos_sympy[2]
phi4=all_rhos_sympy[3]
phi5=all_rhos_sympy[4]
phi6=all_rhos_sympy[5]

In [8]:
phi1+phi2+phi3

In [9]:
phi4+phi5+phi6

In [10]:
num=a*phi3+a*alpha3*phi6
den=phi1+phi2+phi3+phi4+phi5+phi6

In [11]:
dnum=sympy.diff(num,ku)
dden=sympy.diff(den,ku)

In [12]:
left=(sympy.diff(phi3,ku)+alpha3*sympy.diff(phi6,ku))*den
right=(phi3+alpha3*phi6)*dden

In [13]:
B=sympy.symbols('B')

In [14]:
l_r=sympy.expand(left-right)
l_rB=l_r.subs(kb*x,B)
poly_zero=sympy.Poly(l_rB,ku)


In [15]:
poly_zero

In [16]:
for cn,coeff in enumerate(poly_zero.coeffs()[::-1]):
    print(cn)
    display(coeff)

0


1


2


3


4


And it is then easy to proof that if alpha1,alpha2,alpha3>1, each of these coefficients is negative and therefore the derivative is always negative. Meaning an increase in unbinding rate leads to less expression. 

In [48]:
np.random.seed(1)
for i in range(10):
    parset=10**np.random.uniform(-3,3,size=6)
    aval,kuval,kbval,alpha1val,alpha2val,alpha3val=parset
    expr=l_rB.subs({a:aval,B:kbval,ku:kuval,alpha1:alpha1val,alpha2:alpha2val,alpha3:alpha3val})
    print(expr)

5.80698924000057
1.37099209057427e-8
-1.41587230636045
192483732.747763
6.60357117491259e+20
5776641.22696706
-5.93650368039794e+17
-54.8476648266322
1.07062857176161e-12
1208944.40632542


In [51]:
np.random.seed(2)
pos=0
neg=0
for i in range(10000):
    parset=10**np.random.uniform(-3,3,size=3)
    aval,kuval,kbval=parset
    alpha1val,alpha2val,alpha3val=10**np.random.uniform(0,3,size=3)
    expr=l_rB.subs({a:aval,B:kbval,ku:kuval,alpha1:alpha1val,alpha2:alpha2val,alpha3:alpha3val})
    if expr>0:
        pos+=1
    else:
        neg+=1
print("pos",pos,"neg",neg)

pos 0 neg 10000
